This tests the accuracy of a differential operator on a polynomial of the given order. If the operator is working to said order, then the `Actual` and `Theoretical` outputs should match (except possibly at the edges.)

In [1]:
import os.path
from scipy import *
import numpy as np
from numpy import *
from numpy import linalg as LA
from scipy import linalg as LA2
import sympy as sympy
import sys as sys
import time
import matplotlib.pyplot as plt
import itertools as it
from IPython.core.display import HTML

sys.path.append('/Users/sashacurcic/SashasDirectory/ANAG/FV_MG/')
from Modules import BasicTools as BT
from Modules import WaveTools as WT
from Modules import PlotTools as PT
from Modules import FFTTools as FFTT
from Modules import OperatorTools as OT
from Modules import GridTransferTools as GTT
from Modules import TestTools as TT
from Modules import SolverTools as ST

display(HTML("<style>pre { white-space: pre !important; }</style>"))
np.set_printoptions( linewidth = 10000, threshold = 100000)

This cell takes all of the input parameters.

In [2]:
# Grid parameters.
nh_min = 32
refRatio = 2
cfInter = int(nh_min / refRatio)

L = 1
locs = [0.5]
epsilons = [1, 0.5] # [1, 1]
mus = [1, 0.5]

# Time integration parameters.
deriv = 'UD'
RL = 'L'
# RK = 4
orders = 5

This cell contains the relevant switch information.

In [3]:
printOut = False



This cell instantiates the Grid and the Physics objects.

In [4]:
omega = BT.Grid(nh_min)
finepart = list(np.arange(int(nh_min / 4), int(3 * nh_min / 4))) # list(np.arange(int(nh_min / refRatio)))# + int(nh_min / refRatio)) # list(np.arange(int(nh_min / 4), int(3 * nh_min / 4)))
omega.AddPatch(refRatio, finepart)
degFreed = omega.degFreed


physics = BT.PhysProps(omega, epsilons, mus, locs, L)

This cell runs a test on the derivative operator on a cell-averaged polynomial which matches the given order.  
$P (x) = \sum_{k = 0}^{o} x^{k}$  
$<P (x)> = \frac{1}{\Delta x} \int_{x}^{x + \Delta x} P (x')$d$x'$  
$<P (x)> = \frac{1}{\Delta x} \sum_{k = 1}^{o} \frac{1}{k} [(x + \Delta x)^{k} - x^{k}]$  
$<\frac{d P (x)}{d x}> = \sum_{k = 1}^{o} [(x + \Delta x)^{k} - x^{k}]$  
This derivative operator comprises both the left and right cell faces and assumes periodic boundaries. This method of approximating the derivative produces as many nonzero error elements as the order of the approximation at the boundaries of the whole solution space, no matter where the refinement patch is located. It works up to order 9 for upwind and downwind derivatives and up to order 10 for center-difference derivatives.


In [5]:
for o in range(orders):
    order = 2*o + 1
    if ((deriv == 'CD') or (deriv == 'C')):
        order = order + 1
    print('Order:', order)
    TT.DerivPolyTest(omega, deriv, order, deriv = 0, printOut = printOut)

Order: 1
Difference Between Actual and Theoretical:
[32.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
Total Nonzero Error Elements: 1


Order: 3
Difference Between Actual and Theoretical:
[ 78.74479167 -15.58854167  -0.           0.          -0.          -0.           0.          -0.          -0.          -0.           0.          -0.          -0.           0.          -0.          -0.           0.          -0.          -0.           0.          -0.           0.           0.          -0.           0.           0.          -0.           0.           0.          -0.           0.           0.          -0.           0.           0.          -0.          -0.           0.          -0.          -0.           0.          -0.           0.           0.          -0.           0.           0.          32.84375   ]
Total Nonzero Error Elements: 3


Order: 5

This cell runs a similar test, but instead of a derivative, it operates on one face at a time. Then it takes the difference of those faces and divides by the appropriate length intervals. Like the test of the derivative operator, it also assumes periodic boundaries. Unlike in the derivative operator case, in which interpolation and restriction were predicated on the proximity of the central cell of the derivative to the AMR boundary, the face operator determines restriction and interpolation based on the distance of that face from the patch boundary. Because of this, it gains an extra nonzero element located at each patch boundary. So that is one extra error element if the patch is located entirely on one side of the whole solution space and two extra error elements at either side of the patch if it is located in the middle. This pattern is consistent for both upwind and downwind face operators up to order 9. It is not yet set up to handle center-difference faces.


In [6]:
for o in range(orders):
    order = 2*o + 1
    if ((deriv == 'CD') or (deriv == 'C')):
        order = order + 1
    print('Order:', order)

    TT.FacePolyTest1(omega, deriv, order, RL, printOut = printOut)

Order: 1
Difference Between Actual and Theoretical:
[ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0. 32.]
Total Nonzero Error Elements: 1


Order: 3
Difference Between Actual and Theoretical:
[ 31.17708333333  -0.              0.              0.             -0.              0.              0.             -0.             -0.              0.              0.             -0.              0.              0.             -0.              0.              0.             -0.              0.             -0.             -0.              0.             -0.             -0.              0.             -0.             -0.              0.             -0.             -0.              0.             -0.             -0.              0.              0.             -0.              0.              0.             -0.             -0.             -0.             -0.       

This cell runs the same test as the previous cell, except it gets rid of periodic boundaries. The results on the error remain unchanged.


In [7]:
for o in range(orders):
    order = 2*o + 1
    if ((deriv == 'CD') or (deriv == 'C')):
        order = order + 1
    Ng = 0
    print('Order:', order)
    TT.FacePolyTest(omega, physics, deriv, order, RL, Ng, printOut = printOut, pieces = False)

Order: 1
Difference Between Actual and Theoretical:
[ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0. 32.]
Total Nonzero Error Elements: 1


Order: 3
Difference Between Actual and Theoretical:
[ 31.17708333333  -0.              0.              0.             -0.              0.              0.             -0.             -0.              0.              0.             -0.              0.              0.             -0.              0.              0.             -0.              0.             -0.             -0.              0.             -0.             -0.              0.             -0.             -0.              0.             -0.             -0.              0.             -0.             -0.              0.              0.             -0.              0.              0.             -0.             -0.             -0.             -0.       

This cell runs the same test as the previous cell, except it splits the regions in two at the location of the material boundary. The result is that several more error elements are added at the location of the incision.


In [8]:
for o in range(orders):
    order = 2*o + 1
    if ((deriv == 'CD') or (deriv == 'C')):
        order = order + 1
    Ng = 0
    print('Order:', order)
    TT.FacePolyTest(omega, physics, deriv, order, RL, Ng, printOut = printOut, pieces = True)

Order: 1
Difference Between Actual and Theoretical:
[ 0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.  96.5]
[ 0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.  64.5]
Total Nonzero Error Elements: 2


Order: 3
Difference Between Actual and Theoretical:
[-10.50339084201  -0.              0.              0.             -0.              0.              0.             -0.             -0.              0.              0.             -0.              0.              0.             -0.              0.              0.             -0.              0.             -0.             -0.              0.            -20.23134697808 100.68523830838]
[-39.54598659939  -0.             -0.              0.             -0.             -0.              0.             -0.             -0.              0.              0.             -0.              0.              0.             -0.   

This cell runs the same test as the previous cell, except it interpolates order-minus-2-many ghost cells at the material boundary using the function from Hans' Matlab code. The results remain just as bad as the previous cell.


In [9]:
for o in range(orders):
    order = 2*o + 1
    if ((deriv == 'CD') or (deriv == 'C')):
        order = order + 1
        Ng = int(order / 2)
    else:
        Ng = int((order + 1) / 2)
    print('Order:', order)
    TT.FacePolyTest(omega, physics, deriv, order, RL, Ng, printOut = printOut, pieces = True)

Order: 1

Difference Between Actual and Theoretical:
[0.            0.            0.            0.            0.            0.            0.            0.            0.            0.            0.            0.            0.            0.            0.            0.            0.            0.            0.            0.            0.            0.            0.            0.66666666667]
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]
Total Nonzero Error Elements: 2


Order: 3

Difference Between Actual and Theoretical:
[-10.50339084201  -0.              0.              0.             -0.              0.              0.             -0.             -0.              0.              0.             -0.              0.              0.             -0.              0.              0.             -0.              0.             -0.             -0.              0.             -0.0001373291    0.00009155273]
[ 0.0002746582  -0.            -0.             0.            -